# Práctica 2. MT. Sabela Iglesias

### Preparativos

In [12]:
import os
import numpy as np
import spacy
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from scipy.optimize import linear_sum_assignment
import re
import chardet

In [13]:
!git clone https://github.com/sigelsias/master-MT-P2.git

fatal: destination path 'master-MT-P2' already exists and is not an empty directory.


In [14]:
src_dir = '/content/master-MT-P2/Corpus-Clustering'
processed_dir = '/content/master-MT-P2/processed-data'

In [15]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read(10000)
        result = chardet.detect(raw_data)
        return result['encoding']

def count_words(file_path):
    try:
        encoding = detect_encoding(file_path)
        with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
            text = file.read()
            words = text.split()
            return len(words)
    except Exception as e:
        print(f"Error al procesar el archivo {file_path}: {e}")
        return 0

def describe_collection(root_path):
    if not os.path.exists(root_path):
        print(f"La ruta {root_path} no existe.")
        return

    stats_by_group = {}

    for group_name in os.listdir(root_path):
        group_path = os.path.join(root_path, group_name)

        if os.path.isdir(group_path):

            file_lengths = []
            for filename in os.listdir(group_path):
                file_path = os.path.join(group_path, filename)
                if os.path.isfile(file_path):
                    word_count = count_words(file_path)
                    file_lengths.append(word_count)

            if file_lengths:
                num_documents = len(file_lengths)
                mean_words = np.mean(file_lengths)
                std_words = np.std(file_lengths)
            else:
                num_documents = 0
                mean_words = 0
                std_words = 0

            stats_by_group[group_name] = {
                'num_documents': num_documents,
                'mean_words': mean_words,
                'std_words': std_words
            }

    # Mostrar los resultados
    for group, stats in stats_by_group.items():
        print(f"Grupo: {group}")
        print(f"Número de documentos: {stats['num_documents']}")
        print(f"Número medio de palabras: {stats['mean_words']:.2f}")
        print(f"Desviación estándar de palabras: {stats['std_words']:.2f}")
        print("-" * 50)

describe_collection(src_dir)


Grupo: talk.politics.mideast
Número de documentos: 72
Número medio de palabras: 726.88
Desviación estándar de palabras: 1321.51
--------------------------------------------------
Grupo: sci.electronics
Número de documentos: 211
Número medio de palabras: 265.78
Desviación estándar de palabras: 807.43
--------------------------------------------------
Grupo: comp.sys.mac.hardware
Número de documentos: 146
Número medio de palabras: 184.38
Desviación estándar de palabras: 108.42
--------------------------------------------------
Grupo: rec.autos
Número de documentos: 61
Número medio de palabras: 216.21
Desviación estándar de palabras: 170.52
--------------------------------------------------
Grupo: rec.sport.hockey
Número de documentos: 50
Número medio de palabras: 273.88
Desviación estándar de palabras: 265.68
--------------------------------------------------
Grupo: comp.sys.ibm.pc.hardware
Número de documentos: 124
Número medio de palabras: 247.70
Desviación estándar de palabras: 373.05

### Preprocesamiento



In [16]:
#! python -m spacy download en

In [17]:
def preprocess_text(text):
    # Eliminar cabeceras (hasta la primera línea en blanco)
    if "\n\n" in text:
        text = text.split("\n\n", 1)[1]

    #Tockenizacion, lematización eliminar stop words
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    lemas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    text= " ".join(lemas)
    processed_text = text.lower()
    return processed_text


In [18]:
def process_and_save_files(src_dir, dest_dir):
    # Crear la carpeta processed-data si no existe
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Recorremos todas las carpetas y archivos dentro de Corpus-Clustering
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            file_path = os.path.join(root, file)

            # Intentamos abrir el archivo con una codificación flexible
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
            except UnicodeDecodeError:
                    with open(file_path, 'r', encoding='ISO-8859-1') as f:
                        text = f.read()

            # Aplicar preprocesamiento
            processed_text = preprocess_text(text)

            # Guardamos un archivo con los textos procesados
            processed_file_path = os.path.join(dest_dir, file)
            with open(processed_file_path, 'w', encoding='utf-8') as f:
                f.write(processed_text)

In [19]:
# process_and_save_files(src_dir, processed_dir)
print("Preprocesamiento completo y archivos guardados en 'processed-data'.")


Preprocesamiento completo y archivos guardados en 'processed-data'.


### Procesar golden standar

In [20]:
class text_class:
    def __init__(self, id, clase):
        self.id = id
        self.clase = clase

    def __repr__(self):
        return f"text_class(id='{self.id}', clase='{self.clase}')"

golden_standard = []

src_dir = "/content/master-MT-P2/Corpus-Clustering"  # Asegúrate de definir la ruta correcta
for root, dirs, files in os.walk(src_dir):
    for file in files:
        file_path = os.path.join(root, file)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
        except UnicodeDecodeError:
            # Si hay un error, intenta con otra codificación (por ejemplo, 'latin-1')
            print(f"Error al leer {file}. Intentando con 'latin-1'.")
            try:
                with open(file_path, 'r', encoding='latin-1') as f:
                    text = f.read()
            except UnicodeDecodeError:
                print(f"No se pudo leer {file} con ninguna codificación.")
                continue  # Si tampoco se puede leer, omite este archivo

        # Procesar el texto para asignar la clase
        if "comp.sys.ibm.pc.hardware" in text:
            text_obj = text_class(id=file, clase="comp.sys.ibm.pc.hardware")
        elif "comp.sys.mac.hardware" in text:
            text_obj = text_class(id=file, clase="comp.sys.mac.hardware")
        elif "rec.autos" in text:
            text_obj = text_class(id=file, clase="rec.autos")
        elif "rec.sport.hockey" in text:
            text_obj = text_class(id=file, clase="rec.sport.hockey")
        elif "sci.electronics" in text:
            text_obj = text_class(id=file, clase="sci.electronics")
        elif "talk.politics.guns" in text:
            text_obj = text_class(id=file, clase="talk.politics.guns")
        elif "talk.politics.mideast" in text:
            text_obj = text_class(id=file, clase="talk.politics.mideast")
        else:
            print(f'Fichero {file} fuera de clases.')
            continue
        golden_standard.append(text_obj)

# Ordenar los objetos por el atributo 'id'
golden_standard = sorted(golden_standard, key=lambda x: x.id)

# Recorrer los objetos ordenados y extraer el atributo clase
golden_labels = [obj.clase for obj in golden_standard]
len(golden_labels)


Error al leer 101596. Intentando con 'latin-1'.


805

### Carga textos procesados


In [21]:
ruta_carpeta = "/content/master-MT-P2/processed-data"
textos = []
for archivo in os.listdir(ruta_carpeta):
      with open(os.path.join(ruta_carpeta, archivo), "r", encoding="utf-8") as f:
          textos.append(f.read())
print(f"Se cargaron {len(textos)} textos.")

Se cargaron 805 textos.


###Evaluación

In [22]:
def evaluateCluster(golden_labels, cluster_labels):
  cluster_labels = [str(label) for label in cluster_labels]
  #Este diccionario será un primer mapeo inicial para traducir las clases a formato numérico y vieversa
  traduccion = {
      0: "comp.sys.ibm.pc.hardware",
      1: "comp.sys.mac.hardware",
      2: "rec.autos",
      3: "rec.sport.hockey",
      4: "sci.electronics",
      5: "talk.politics.guns",
      6: "talk.politics.mideast"
  }

  # Convertir las etiquetas de cluster_labels a las clases reales
  mapped_cluster_labels = [traduccion[int(label)] for label in cluster_labels]

  # Matriz de confusión y obtener la relación de cluster a clase que maximiza la similitud
  conf_matrix = confusion_matrix(golden_labels, mapped_cluster_labels)
  class_ind, cluster_ind = linear_sum_assignment(-conf_matrix)

  # Crear un mapeo entre Cluster y las etiquetas reales
  mapping = {v:k for k, v in zip(class_ind, cluster_ind)}
  mapped_labels = [traduccion[mapping[int(label)]] for label in cluster_labels]

  #Construimos los resultados de la función
  report = classification_report(golden_labels, mapped_labels, output_dict=True)
  cm=confusion_matrix(golden_labels, mapped_labels)
  precision_macro = report['macro avg']['precision']
  recall_macro = report['macro avg']['recall']
  f1_macro = report['macro avg']['f1-score']

  return np.array([precision_macro, recall_macro, f1_macro]), cm

### K-Means

In [23]:

vectorizer = CountVectorizer()
X_tf = vectorizer.fit_transform(textos)
# Aplicar K-means con 7 clusters
kmeans1 = KMeans(n_clusters=7, random_state=42)
kmeans1.fit(X_tf)

#Evaluacion
evaluateCluster(golden_labels,kmeans1.labels_)

(array([0.6084855 , 0.14911254, 0.07247422]),
 array([[  0,   0,   0,   0, 127,   0,   0],
        [  0,   1,   0,   0, 142,   0,   0],
        [  0,   0,   0,   0,  64,   0,   0],
        [  1,   0,   1,   1,  47,   0,   0],
        [  0,   0,   1,   0, 207,   0,   0],
        [  0,   0,   0,   0, 144,   1,   0],
        [  0,   0,   0,   0,  67,   0,   1]]))

In [24]:
vectorizer = TfidfVectorizer(
    max_df=400,
    min_df=20,
    stop_words="english",
)
X_tfidf = vectorizer.fit_transform(textos)
kmeans2 = KMeans(n_clusters=7, random_state=42)
kmeans2.fit(X_tfidf)
evaluateCluster(golden_labels,kmeans2.labels_)

(array([0.17323868, 0.17786142, 0.17293659]),
 array([[26, 28,  8, 16, 23, 18,  8],
        [20, 43, 10, 11, 26, 24,  9],
        [ 7, 16,  6,  7,  8, 15,  5],
        [11,  9,  2,  6,  9,  6,  7],
        [38, 42, 21, 11, 37, 29, 30],
        [24, 31, 13, 13, 24, 27, 13],
        [10, 15,  3,  8, 14,  7, 11]]))

In [25]:
# Crear el vectorizador con binarización
vectorizer = CountVectorizer(binary=True, stop_words='english')

# Ajustar y transformar el conjunto de datos de texto
X_bin = vectorizer.fit_transform(textos)
kmeans3 = KMeans(n_clusters=7, random_state=42)
kmeans3.fit(X_bin)

KMeans(n_clusters=7, random_state=42)

In [26]:
cluster_labels_km3=kmeans3.labels_
evaluateCluster(golden_labels,cluster_labels_km3)

(array([0.41565305, 0.14638746, 0.07373109]),
 array([[  2,   0,   0,   0, 125,   0,   0],
        [  1,   1,   0,   0, 141,   0,   0],
        [  0,   0,   1,   0,  62,   1,   0],
        [  1,   0,   0,   1,  47,   0,   1],
        [  6,   0,   0,   1, 201,   0,   0],
        [  1,   0,   0,   0, 144,   0,   0],
        [  2,   0,   0,   0,  66,   0,   0]]))

### Modelo Aglomerativo

In [27]:
modelo = AgglomerativeClustering(n_clusters=7, linkage='ward')
X_tf = X_tf.toarray()
cluster_labels_ag1 = modelo.fit_predict(X_tf)
evaluateCluster(golden_labels,cluster_labels_ag1)

(array([0.39763044, 0.14658294, 0.07615588]),
 array([[  1,   0,   0,   0, 123,   3,   0],
        [  0,   1,   0,   0, 139,   3,   0],
        [  0,   0,   0,   0,  60,   4,   0],
        [  1,   0,   0,   1,  45,   2,   1],
        [  1,   0,   1,   0, 199,   7,   0],
        [  0,   0,   0,   0, 140,   5,   0],
        [  0,   0,   0,   0,  66,   2,   0]]))

In [28]:
X_tfidf=X_tfidf.toarray()
cluster_labels_ag2 = modelo.fit_predict(X_tfidf)
evaluateCluster(golden_labels,cluster_labels_ag2)

(array([0.20150647, 0.16759464, 0.14762503]),
 array([[ 28,  26,   0,   0,  70,   2,   1],
        [ 22,  29,   2,   2,  82,   2,   4],
        [ 10,  13,   1,   1,  38,   1,   0],
        [  9,   6,   1,   2,  30,   1,   1],
        [ 32,  31,   4,   5, 125,   8,   3],
        [ 23,  19,   3,   1,  89,   5,   5],
        [ 14,   7,   1,   2,  40,   0,   4]]))

In [29]:
X_bin=X_bin.toarray()
cluster_labels_ag3 = modelo.fit_predict(X_bin)
evaluateCluster(golden_labels,cluster_labels_ag3)

(array([0.53659815, 0.16099927, 0.10238742]),
 array([[ 24,   0,   1,   0, 102,   0,   0],
        [ 16,   1,   0,   0, 126,   0,   0],
        [  8,   0,   0,   0,  56,   0,   0],
        [  8,   0,   1,   1,  39,   1,   0],
        [ 20,   0,   1,   0, 185,   2,   0],
        [ 12,   0,   0,   0, 132,   1,   0],
        [ 12,   0,   0,   0,  55,   0,   1]]))